In [1]:
import nltk
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics 
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import permutation_test_score
import re
import random
from sklearn.linear_model import LogisticRegression

/Users/fatma/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /Users/fatma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " ", string)
    string = re.sub(r"\'ve", " ", string)
    string = re.sub(r"n\'t", " ", string)
    string = re.sub(r"n\'s", " ", string)
    string = re.sub(r"\'re", " ", string)
    string = re.sub(r"\'d", " ", string)
    string = re.sub(r"\'ll", " ", string)
    string = re.sub(r",", " ", string)
    string = re.sub(r"!", " ", string)
    string = re.sub(r"\(", " ", string)
    string = re.sub(r"\)", " ", string)
    string = re.sub(r"\?", " ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"\//?", " ", string)
    string = re.sub(r"\d+", " ", string)
    string = re.sub(r"\$", " ", string)
    string = re.sub(r"\#", " ", string)
    return string.strip().lower()

In [4]:
def remove_stop(str):
    stop = set(stopwords.words('english'))
    lst = str.split()
    lst = [i for i in lst if i not in stop]
    return ' '.join(lst)

def stem (str):
    lst = str.split()
    lst = [st.stem(x) for x in lst]
    return ' '.join(lst)

def remove_unwanted_words(str):
    unwanted_words = ["httpaddress", "usrid", "D", "dd", "rt", "amp", "am", "pm", '``',
                      "''", "", "//", "\\", "\\'s", "\\?", "\?","http","httpaddresshttpaddresst", "cohttpaddressek",
                      "taksim","gezi", "park", "direngeziparki", "occupygezi", "istanbul", "turkish","turkey",
                      "protest","direngezipark","direnankara","geziparki", "protesters", "protests", "sat", "sun", "mon,",
                     "tue", "wed", "thu", "fri", "jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
    lst = str.split(" ")
    lst = [i for i in lst if i not in unwanted_words]
    return ' '.join(lst)

def toLower(str):
    lst = str.split()
    lst = [i.lower() for i in lst]
    return ' '.join(lst)

def word_len (str):
    lst = str.split()
    lst = [i for i in lst if len(i)>1 and len(i) <7]
    return ' '.join(lst)

def sent_len (str):
    lst = str.split()
    if len(lst)>=3:
        return ' '.join(lst)

In [63]:
def clean_data(data):
    """
    Loads MR polarity african_data from files, splits the african_data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load african_data from files
    positive_examples = data
    positive_examples = positive_examples.strip() 
    positive_examples = remove_stop(positive_examples) 
    #print(positive_examples[0])
    positive_examples = toLower(positive_examples) 
    #rint(positive_examples[0])
    positive_examples = clean_str(positive_examples) 
    #rint(positive_examples[0])
    #positive_examples = [stem(item) for item in positive_examples]
    #rint(positive_examples[0])
    positive_examples = remove_unwanted_words(positive_examples)
    #positive_examples = word_len(positive_examples) 
    #positive_examples = sent_len(positive_examples) 
    
    return positive_examples
    

In [99]:
df_cf_judgments = pd.read_csv('Data/CF_combined_aggreagated_tasks_21_05_2018.csv')

In [100]:
df_cf_judgments.columns

Index(['Unnamed: 0', '_unit_id', '_golden', '_trusted_judgments', 'violence',
       'violence_confidence', 'violence_gold', 'protest', 'protest_confidence',
       'protest_gold', 'id', 'text', 'created_at'],
      dtype='object')

In [101]:
len(df_cf_judgments)

1197

In [102]:
len(df_cf_judgments.text.drop_duplicates())

1030

In [110]:
df_cf_judgments.drop_duplicates()

,Unnamed: 0,_unit_id,_golden,_trusted_judgments,violence,violence_confidence,violence_gold,protest,protest_confidence,protest_gold,id,text,created_at,clean_text
0,0,1285876452,False,3,0,0.0000,NaN,0,0.6667,NaN,340810574529171000,RT @ekizilkaya: RT @mehmetbinay Another powerf...,Sat Jun 01 12:42:27 +0000 2013,ekizilkaya mehmetbinay another powerful pic t ...
1,1,1285876453,False,3,1,1.0000,NaN,1,1.0000,NaN,340712838953853000,A Letter to the Rest of the World http://t.co/...,Sat Jun 01 06:14:05 +0000 2013,a letter rest world t co ssl ksveib ireport sa...
2,2,1285876454,False,3,2,NaN,NaN,0,1.0000,NaN,340730727110939000,Fresh anti-government clashes hit Turkey - Eur...,Sat Jun 01 07:25:10 +0000 2013,fresh anti government clashes hit europe al ja...
3,3,1285876455,False,5,3,NaN,NaN,0,1.0000,NaN,346087667177509000,ma_milsoni #russia#russian#jamaica#jamaican#uk...,Sun Jun 16 02:11:44 +0000 2013,ma milsoni russia russian jamaica jamaican ukr...
4,4,1285876456,False,3,4,NaN,NaN,0,1.0000,NaN,346803250600767000,There's a #Turkey flag on #Pakistan 's note of...,Tue Jun 18 01:35:12 +0000 2013,there flag pakistan note rupees thousand
5,5,1285876457,False,3,5,NaN,NaN,0,1.0000,NaN,341715297901883000,"RT @melissarfleming: Meanwhile, in #Turkey, th...",Tue Jun 04 00:37:30 +0000 2013,melissarfleming meanwhile syrian refugees wome...
6,6,1285876458,False,3,6,1.0000,NaN,1,1.0000,NaN,341166068213105000,RT @sneijder101010: My heart is with Turkish p...,Sun Jun 02 12:15:03 +0000 2013,sneijder my heart people defending rights
7,7,1285876459,False,4,7,NaN,NaN,0,1.0000,NaN,343227500446482000,"""As It Fights in #Syria, #Hezbollah Seen Unfav...",Sat Jun 08 04:46:27 +0000 2013,as it fights syria hezbollah seen unfavorably ...
8,8,1285876460,False,3,8,NaN,NaN,0,1.0000,NaN,340816781356696000,"RT @ClaireBerlinski: ""It is time for the inter...",Sat Jun 01 13:07:07 +0000 2013,claireberlinski it time international communit...
9,9,1285876461,False,3,9,NaN,NaN,0,1.0000,NaN,341080707285783000,RT @MahirZeynalov: RT @IvanCNN Judging how muc...,Sun Jun 02 06:35:52 +0000 2013,mahirzeynalov ivancnn judging much beer drunk ...


In [113]:
df_cf_judgments.count()

Unnamed: 0             1197
_unit_id               1197
_golden                1197
_trusted_judgments     1197
violence               1195
violence_confidence     683
violence_gold            47
protest                1197
protest_confidence     1197
protest_gold            101
id                     1197
text                   1197
created_at             1197
clean_text             1197
dtype: int64

In [114]:
df_cf_judgments.columns

Index(['Unnamed: 0', '_unit_id', '_golden', '_trusted_judgments', 'violence',
       'violence_confidence', 'violence_gold', 'protest', 'protest_confidence',
       'protest_gold', 'id', 'text', 'created_at', 'clean_text'],
      dtype='object')

In [115]:
df_cf_judgments['violence'] = df_cf_judgments[df_cf_judgments['violence']!=-1]

In [116]:
df_cf_judgments['violence'] = df_cf_judgments['violence'].fillna(0)

In [117]:
df_cf_judgments['clean_text'] = df_cf_judgments['text'].apply(lambda x:clean_data(x) )

In [118]:
len(df_cf_judgments)

1197

In [95]:
df_cf_judgments

,Unnamed: 0,_unit_id,_golden,_trusted_judgments,violence,violence_confidence,violence_gold,protest,protest_confidence,protest_gold,id,text,created_at,clean_text
0,0,1285876452,False,3,0,0.0000,NaN,0,0.6667,NaN,340810574529171000,RT @ekizilkaya: RT @mehmetbinay Another powerf...,Sat Jun 01 12:42:27 +0000 2013,ekizilkaya mehmetbinay another powerful pic t ...
1,1,1285876453,False,3,1,1.0000,NaN,1,1.0000,NaN,340712838953853000,A Letter to the Rest of the World http://t.co/...,Sat Jun 01 06:14:05 +0000 2013,a letter rest world t co ssl ksveib ireport sa...
2,2,1285876454,False,3,2,NaN,NaN,0,1.0000,NaN,340730727110939000,Fresh anti-government clashes hit Turkey - Eur...,Sat Jun 01 07:25:10 +0000 2013,fresh anti government clashes hit europe al ja...
3,3,1285876455,False,5,3,NaN,NaN,0,1.0000,NaN,346087667177509000,ma_milsoni #russia#russian#jamaica#jamaican#uk...,Sun Jun 16 02:11:44 +0000 2013,ma milsoni russia russian jamaica jamaican ukr...
4,4,1285876456,False,3,4,NaN,NaN,0,1.0000,NaN,346803250600767000,There's a #Turkey flag on #Pakistan 's note of...,Tue Jun 18 01:35:12 +0000 2013,there flag pakistan note rupees thousand
5,5,1285876457,False,3,5,NaN,NaN,0,1.0000,NaN,341715297901883000,"RT @melissarfleming: Meanwhile, in #Turkey, th...",Tue Jun 04 00:37:30 +0000 2013,melissarfleming meanwhile syrian refugees wome...
6,6,1285876458,False,3,6,1.0000,NaN,1,1.0000,NaN,341166068213105000,RT @sneijder101010: My heart is with Turkish p...,Sun Jun 02 12:15:03 +0000 2013,sneijder my heart people defending rights
7,7,1285876459,False,4,7,NaN,NaN,0,1.0000,NaN,343227500446482000,"""As It Fights in #Syria, #Hezbollah Seen Unfav...",Sat Jun 08 04:46:27 +0000 2013,as it fights syria hezbollah seen unfavorably ...
8,8,1285876460,False,3,8,NaN,NaN,0,1.0000,NaN,340816781356696000,"RT @ClaireBerlinski: ""It is time for the inter...",Sat Jun 01 13:07:07 +0000 2013,claireberlinski it time international communit...
9,9,1285876461,False,3,9,NaN,NaN,0,1.0000,NaN,341080707285783000,RT @MahirZeynalov: RT @IvanCNN Judging how muc...,Sun Jun 02 06:35:52 +0000 2013,mahirzeynalov ivancnn judging much beer drunk ...


In [96]:
len(df_cf_judgments)

1197

In [97]:
df_cf_judgments.columns

Index(['Unnamed: 0', '_unit_id', '_golden', '_trusted_judgments', 'violence',
       'violence_confidence', 'violence_gold', 'protest', 'protest_confidence',
       'protest_gold', 'id', 'text', 'created_at', 'clean_text'],
      dtype='object')

In [98]:
test_predict_auc_all =[]
for i in range(1,10):
    df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df_cf_judgments.clean_text, 
                                                    df_cf_judgments.violence, 
                                                    test_size=0.5)
    df_train_df = pd.concat([df_X_train,df_y_train],axis=1)
    df_test_df = pd.concat([df_X_test,df_y_test],axis = 1)
    print('training set size', len(df_train_df))
    print('training pos', len(df_train_df[df_train_df["violence"]==1]))
    print('training neg', len(df_train_df[df_train_df["violence"]==0]))
    print('test set size', len(df_test_df))
    print('test pos', len(df_test_df[df_test_df["violence"]==1]))
    print('test neg', len(df_test_df[df_test_df["violence"]==0]))
    
    df_pipeline_violence_Svc = Pipeline([
    ('vectorizer',  CountVectorizer(ngram_range=(1, 2),stop_words='english',  min_df=3)),
    ('classifier',  SVC(kernel='linear', probability=True)) ])

    
    df_scores = cross_val_score(df_pipeline_violence_Svc, df_train_df.clean_text, df_train_df.violence, cv=10)
    print('scores',df_scores)
    print('scores mean',df_scores.mean())
    print('scores std',df_scores.std() * 2)
    
    df_cv_predict = cross_val_predict(df_pipeline_violence_Svc, df_train_df.clean_text, df_train_df.violence, cv=10)
    df_cv_predict_scores = metrics.roc_auc_score(df_train_df.violence, df_cv_predict)
    print('cv auc', df_cv_predict_scores)
    
    df_pipeline_violence_Svc_model = df_pipeline_violence_Svc.fit(df_train_df.clean_text, df_train_df.violence)
    df_prediction = df_pipeline_violence_Svc_model.predict(df_test_df.clean_text)
    df_predict_prob = df_pipeline_violence_Svc_model.predict_proba(df_test_df.clean_text)
    test_predict_auc = metrics.roc_auc_score(df_test_df.violence,df_prediction)
    print('test set auc', test_predict_auc)
    test_predict_auc_all.append(test_predict_auc)
    print(metrics.confusion_matrix(df_test_df.violence,df_prediction))

training set size 598
training pos 1
training neg 0
test set size 599
test pos 0
test neg 1


ValueError: Unknown label type: 'unknown'